In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
def preprocess(text):
    return text.lower().strip()
df['review']=df['review'].apply(preprocess)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.transform(y_test)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq=tokenizer.texts_to_sequences(X_train)
X_test_seq=tokenizer.texts_to_sequences(X_test)

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len=100
X_train_pad=pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad=pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
model=Sequential([
    Embedding(5000, 128, input_length=max_len),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_pad, y_train, batch_size=128, epochs=10)

Epoch 1/10
313/313 [==============================] - 67s 195ms/step - loss: 0.3043 - accuracy: 0.8760
Epoch 2/10
313/313 [==============================] - 60s 192ms/step - loss: 0.2499 - accuracy: 0.8993
Epoch 3/10
313/313 [==============================] - 60s 193ms/step - loss: 0.2144 - accuracy: 0.9159
Epoch 4/10
313/313 [==============================] - 60s 193ms/step - loss: 0.1859 - accuracy: 0.9270
Epoch 5/10
313/313 [==============================] - 65s 208ms/step - loss: 0.1632 - accuracy: 0.9367
Epoch 6/10
313/313 [==============================] - 64s 203ms/step - loss: 0.1383 - accuracy: 0.9498
Epoch 7/10
313/313 [==============================] - 62s 199ms/step - loss: 0.1155 - accuracy: 0.9589
Epoch 8/10
313/313 [==============================] - 61s 196ms/step - loss: 0.1080 - accuracy: 0.9617
Epoch 9/10
313/313 [==============================] - 61s 194ms/step - loss: 0.0875 - accuracy: 0.9701
Epoch 10/10
313/313 [==============================] - 61s 196ms/step - l

In [12]:
loss, acc=model.evaluate(X_test_pad, y_test)
loss, acc

313/313 [==============================] - 12s 34ms/step - loss: 0.5496 - accuracy: 0.8506


(0.5495979189872742, 0.850600004196167)

In [17]:
sample="The movie was bad"
sample_seq=tokenizer.texts_to_sequences([sample])
sample_pad=pad_sequences(sample_seq, maxlen=max_len, padding='post')
prediction=model.predict(sample_pad)
pred=(prediction>0.5).astype(int)[0][0]
pred_label=encoder.inverse_transform([pred])[0]
print(pred_label)

1/1 [==============================] - 0s 62ms/step
negative
